In [1]:
import pandas as pd
import numpy as np
import os 
import shutil 
from nilearn import image
import fsl.wrappers as fsl

In [15]:
df = pd.read_csv('pair_cor_fdr.csv')
#df = df.drop('Unnamed: 0', axis=1)
#df = df.drop('Unnamed: 0.1', axis=1)
df

,region,sws_total
0,zfALFF_17Networks_9,0.193462
1,zfALFF_17Networks_2,0.182355
2,zfALFF_lPFtha_L,0.192590
3,zfALFF_cTtha_L,0.182097
4,zfALFF_Otha_L,0.204450
...,...,...
179,zDCwei_17Networks_LH_SalVentAttnA_FrMed_1,-0.258578
180,zDCwei_17Networks_LH_SalVentAttnA_ParMed_1,-0.203535
181,zDCwei_17Networks_LH_SalVentAttnA_ParOper_1,-0.184510
182,zDCwei_17Networks_LH_DorsAttnB_PostC_1,-0.219417


In [3]:
df["Measurment"] = [meas.split("_")[0] for meas in df["region"]]

In [4]:
df["region"] = ["_".join(meas.split("_")[1:]) for meas in df["region"]]

In [5]:
sch_regions = pd.read_csv("Schaefer2018_400Parcels_17Networks_order_FSLMNI152_1mm.Centroid_RAS.csv").set_index("ROI Name")

In [6]:
BN_regions = pd.read_csv('BN_Atlas_246_LUT_text.csv').set_index("ROI_Name")
BN_regions.head()

,Value
ROI_Name,
mAmyg_L,211
mAmyg_R,212
lAmyg_L,213
lAmyg_R,214
rHipp_L,215


In [7]:
Buck_regions = pd.read_csv('atl-Buckner17.csv')
Buck_regions['name'] = Buck_regions['name'].str.replace(r"Network", "17Networks")
Buck_regions = Buck_regions.set_index('name')
Buck_regions.head()

,index,color
name,,
17Networks_1,1,#b52bd4
17Networks_2,2,#ff0000
17Networks_3,3,#a1e1ff
17Networks_4,4,#6cffc2
17Networks_5,5,#4ac33b


## ReHo

In [8]:
reho = df[df["Measurment"]=="zReHo"]
reho.shape

(18, 3)

In [9]:
for i, region in enumerate(reho["region"]):
    if (region.find("17Network") != -1) and ( len(region) != 13 ):
        value = sch_regions.loc[[region], "ROI Label"].values[0]
        name = 'reho_subregions/'+region+'.nii.gz'
        imp = reho.iloc[[i], 1].values[0]*100
        fsl.fslmaths('Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2mm.nii').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
        
    if (region.find("17Network") == -1):
        value = BN_regions.loc[[region], "Value"].values[0]
        name = 'reho_subregions/'+region+'.nii.gz'
        imp = reho.iloc[[i], 1].values[0]*100
        fsl.fslmaths('BN_Atlas_246_2mm.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
                
    if (region.find("17Network") != -1) and ( len(region) == 13 ):
        value = Buck_regions.loc[[region], "index"].values[0]
        name = 'reho_subregions/'+region+'.nii.gz'
        imp = reho.iloc[[i], 1].values[0]*100
        fsl.fslmaths('Buckner_atlas_resized.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
        
   

## fALFF

In [10]:
fALFF = df[df["Measurment"]=="zfALFF"]
fALFF

,region,sws_total,Measurment
0,17Networks_9,0.193462,zfALFF
1,17Networks_2,0.182355,zfALFF
2,lPFtha_L,0.192590,zfALFF
3,cTtha_L,0.182097,zfALFF
4,Otha_L,0.204450,zfALFF
...,...,...,...
120,17Networks_LH_SomMotB_Aud_3,0.193633,zfALFF
121,17Networks_LH_SomMotB_S2_2,0.246855,zfALFF
122,17Networks_LH_SomMotB_Aud_1,0.281534,zfALFF
123,17Networks_LH_SomMotA_1,0.206542,zfALFF


In [11]:
for i, region in enumerate(fALFF["region"]):
    if (region.find("17Network") != -1) and ( len(region) != 12 ):
        value = sch_regions.loc[[region], "ROI Label"].values[0]
        name = 'fALFF_subregions/'+region+'.nii.gz'
        imp = fALFF.iloc[[i], 1].values[0]*100
        fsl.fslmaths('Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2mm.nii').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
        
    if (region.find("17Network") == -1):
        value = BN_regions.loc[[region], "Value"].values[0]
        name = 'fALFF_subregions/'+region+'.nii.gz'
        imp = fALFF.iloc[[i], 1].values[0]*100
        fsl.fslmaths('BN_Atlas_246_2mm.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
                
    if (region.find("17Network") != -1) and ( len(region) == 12 ):
        value = Buck_regions.loc[[region], "index"].values[0]
        name = 'fALFF_subregions/'+region+'.nii.gz'
        imp = fALFF.iloc[[i], 1].values[0]*100
        fsl.fslmaths('Buckner_atlas_resized.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
        
   

## DC

In [12]:
DC = df[df["Measurment"]=="zDCwei"]
DC

,region,sws_total,Measurment
157,17Networks_9,0.191636,zDCwei
158,lPFtha_L,0.218699,zDCwei
159,mPMtha_L,0.224121,zDCwei
160,dlPu_L,0.203923,zDCwei
161,17Networks_RH_TempPar_5,-0.183178,zDCwei
162,17Networks_RH_TempPar_4,-0.284177,zDCwei
163,17Networks_RH_ContA_PFCl_5,-0.224166,zDCwei
164,17Networks_RH_SalVentAttnA_FrMed_4,-0.204026,zDCwei
165,17Networks_RH_SalVentAttnA_ParMed_4,-0.191214,zDCwei
166,17Networks_RH_SalVentAttnA_ParMed_2,-0.240821,zDCwei


In [13]:
#DC['region'] = DC['region'].str.replace(r"1717Networkss", "17Networks")
DC = DC.iloc[::-1]
DC

,region,sws_total,Measurment
183,17Networks_LH_DorsAttnA_TempOcc_2,-0.190194,zDCwei
182,17Networks_LH_DorsAttnB_PostC_1,-0.219417,zDCwei
181,17Networks_LH_SalVentAttnA_ParOper_1,-0.184510,zDCwei
180,17Networks_LH_SalVentAttnA_ParMed_1,-0.203535,zDCwei
179,17Networks_LH_SalVentAttnA_FrMed_1,-0.258578,zDCwei
178,17Networks_LH_SalVentAttnA_FrMed_2,-0.197819,zDCwei
177,17Networks_LH_ContB_Temp_2,-0.203438,zDCwei
176,17Networks_LH_DefaultB_Temp_5,-0.183486,zDCwei
175,17Networks_LH_DefaultC_PHC_3,-0.204419,zDCwei
174,17Networks_RH_SomMotA_6,-0.216580,zDCwei


In [14]:
for i, region in enumerate(DC["region"]):
    if (region.find("17Network") != -1) and ( len(region) != 12 ):
        value = sch_regions.loc[[region], "ROI Label"].values[0]
        name = 'DC_subregions/'+region+'.nii.gz'
        imp = DC.iloc[[i], 1].values[0]*100
        fsl.fslmaths('Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2mm.nii').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
        
    if (region.find("17Network") == -1):
        value = BN_regions.loc[[region], "Value"].values[0]
        name = 'DC_subregions/'+region+'.nii.gz'
        imp = DC.iloc[[i], 1].values[0]*100
        fsl.fslmaths('BN_Atlas_246_2mm.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
                
    if (region.find("17Network") != -1) and ( len(region) == 12 ):
        reg = "Network_" + region.split("_")[1]
        value = Buck_regions.loc[[reg], "index"].values[0]
        name = 'DC_subregions/'+reg+'.nii.gz'
        imp = DC.iloc[[i], 1].values[0]*100
        fsl.fslmaths('Buckner_atlas_resized.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
           

KeyError: "None of [Index(['Network_9'], dtype='object', name='name')] are in the [index]"

## gmd

In [ ]:
gmd = df[df["Measurment"]=="gmd"]
gmd.shape

In [ ]:
for i, region in enumerate(gmd["region"]):
    if (region.find("17Network") != -1) and ( len(region) != 12 ):
        value = sch_regions.loc[[region], "ROI Label"].values[0]
        name = 'gmd_subregions/'+region+'.nii.gz'
        imp = gmd.iloc[[i], 1].values[0]*100
        fsl.fslmaths('Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2mm.nii').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
        
    if (region.find("17Network") == -1):
        value = BN_regions.loc[[region], "Value"].values[0]
        name = 'gmd_subregions/'+region+'.nii.gz'
        imp = gmd.iloc[[i], 1].values[0]*100
        fsl.fslmaths('BN_Atlas_246_2mm.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
                
    if (region.find("17Network") != -1) and ( len(region) == 12 ):
        reg = "Network_" + region.split("_")[1]
        value = Buck_regions.loc[[reg], "index"].values[0]
        name = 'gmd_subregions/'+reg+'.nii.gz'
        imp = gmd.iloc[[i], 1].values[0]*100
        fsl.fslmaths('Buckner_atlas_resized.nii.gz').thr(value-0.1).uthr(value+0.1).run(name)
        fsl.fslmaths(name).bin().mul(imp).run(name)
        
        

## summation of the parcels to create a mask

In [ ]:
regs = os.listdir("reho_subregions")
for i, reg in enumerate(regs):
    if i != 0:
        source = image.load_img("reho_subregions/reho.nii.gz")
    reg_img = image.load_img("reho_subregions/"+reg)
    if i == 0:
        source = reg_img
    fsl.fslmaths(source).add(reg_img).run("reho_subregions/reho.nii.gz")

In [ ]:
regs = os.listdir("fALFF_subregions")
for i, reg in enumerate(regs):
    if i != 0:
        source = image.load_img("fALFF_subregions/fALFF.nii.gz")
    reg_img = image.load_img("fALFF_subregions/"+reg)
    if i == 0:
        source = reg_img
    fsl.fslmaths(source).add(reg_img).run("fALFF_subregions/fALFF.nii.gz")

In [ ]:
regs = os.listdir("DC_subregions")
for i, reg in enumerate(regs):
    if i != 0:
        source = image.load_img("DC_subregions/DC.nii.gz")
    reg_img = image.load_img("DC_subregions/"+reg)
    if i == 0:
        source = reg_img
    fsl.fslmaths(source).add(reg_img).run("DC_subregions/DC.nii.gz")

In [ ]:
regs = os.listdir("gmd_subregions")
for i, reg in enumerate(regs):
    if i != 0:
        source = image.load_img("gmd_subregions/gmd.nii.gz")
    reg_img = image.load_img("gmd_subregions/"+reg)
    if i == 0:
        source = reg_img
    fsl.fslmaths(source).add(reg_img).run("gmd_subregions/gmd.nii.gz")